In [1]:
from scipy.stats import norm
from time import time
import random    
import pandas as pd

# -1) Cycliste

In [2]:
class Cycliste:
    
    def log_cycliste(self):
        to_print = "%s,%s,%s,%s,%s,%s,%s\n"%(self.identifiant ,
                                        self.sportif     ,
                                        self.age         ,
                                        self.sexe        ,
                                        self.nb_km       ,
                                        self.vitesse     ,
                                         self.attente )
        with open("./logs/cyclistes.csv", "a") as f:
            f.write(to_print)
            
    def log_prise(self, action="prise"):
        import time
        path     = "./logs/prise_%s.txt"%self.identifiant
        path     = "./logs/prises.csv"
        to_print = "%s,%s,%s,%s\n"%(    self.identifiant,
                                        self.heure_de_depart, 
                                        self.velo.nom,
                                        action)
        with open(path, "a") as log_prise:
            log_prise.write(to_print)
            
    def log_rendu(self, action="prise"):
        import time
        path     = "./logs/rendu_%s.txt"%self.identifiant
        path     = "./logs/rendus.csv"
        to_print = "%s,%s,%s,%s,%s\n"%(self.identifiant, 
                                    self.heure_de_depart, 
                                    self.heure_d_arrivee, 
                                    self.nb_km_trajet, 
                                    self.velo.nom )

        with open(path, "a") as log_rendu:
            log_rendu.write(to_print)

            
        
    def __init__(self, identifiant):
        import time
        from scipy.stats import norm
        import random
        homme   = "homme"
        femme   = "femme"
        age_min = 15
        age_max = 80
        sexe    = random.choice([homme, femme])
        age     = random.randint(age_min ,age_max)
        sportif = random.choice([-0.5, 0, 2, 4, 6 ])
        if age < 35:
            facteur = norm( loc =1.2, scale=0.2 )
        elif age <50:
            facteur = norm( loc =1  , scale=0.2 )
        elif age <90:
            facteur = norm( loc =0.7, scale=0.2 )

        if sexe == homme:
            vitesse_de_base = 20
            nb_km = norm( loc =7, scale=2 )
        else:
            vitesse_de_base = 15
            nb_km = norm( loc =10, scale=2 )
        vitesse_moyenne = vitesse_de_base * facteur.rvs(1) + sportif
        
        self.sportif     = sportif
        self.age         = age
        self.sexe        = sexe
        self.nb_km       = nb_km.rvs(1)[0]
        self.vitesse     = vitesse_moyenne[0]
        self.sur_velo    = False
        self.debug       = False
        self.nb_trajet   = 0
        self.durees      = []
        self.trajets     = []
        self.velo        = False
        self.identifiant = "cycliste_%s"%identifiant
        self.prises      = []
        self.attente        = random.randint(0,100)
        self.last_rendu     = time.time()
        
        self.log_cycliste()
        

    def prend_velo(self, station):
        import time
        if time.time() < self.attente + self.last_rendu:
            if self.debug : print "trop tôt (%s Vs %s)"% (time.time() , self.attente + self.last_rendu)
            #print "-",
            return
        if self.sur_velo:
            if self.debug : print "deja sur velo",
            pass
        else:
            import time
            velo = station.prendre_velo()
            if velo:
                velo.disponible      = False
                self.sur_velo        = True
                self.heure_de_depart = time.time()
                self.velo            = velo
                prise = (self.heure_de_depart, self.identifiant, velo.nom )
                self.log_prise()
                try:
                    nb_km_local          = norm( loc =self.nb_km, scale=self.nb_km/float(3) ).rvs(1)[0]
                except Exception as e:
                    print "pbm sur nb_km_local = self.nb_km = %s " %self.nb_km
                    print e
                    nb_km_local = 15
                try:
                    vitesse_local        = norm(loc = self.vitesse, scale = self.vitesse/float(10)).rvs(1)[0]
                    vitesse_local        = vitesse_local * velo.performance
                except:
                    print "pbm sur vitesse_local"
                    print "self.vitesse = %s"%self.vitesse
                    vitesse_local = self.vitesse
                self.nb_km_trajet    = nb_km_local
                duree                = nb_km_local / vitesse_local
                
                
                
                self.heure_d_arrivee =  self.heure_de_depart + duree
                if self.debug : print "arrivee dans %s a %s"%(duree, self.heure_d_arrivee),
                if velo.performance <0.1:
                    duree                = 0
                    self.durees.append( duree)
                    self.heure_d_arrivee =  self.heure_de_depart
                    self.nb_km_trajet    = 0
                    self.rendre_velo(station)
                elif velo.performance <0.3:
                    duree                = duree / 10
                    self.durees.append( duree)
                    self.heure_d_arrivee =  self.heure_de_depart + duree
                    self.nb_km_trajet    = self.nb_km_trajet / float(10)
                    self.rendre_velo(station)
                elif velo.performance <0.5:
                    duree                = duree / 5
                    self.durees.append( duree)
                    self.nb_km_trajet    = self.nb_km_trajet / float(5)
                    self.heure_d_arrivee =  self.heure_de_depart + duree 
                else:
                    self.durees.append( duree)
                
                    
            else:
                if self.debug : print "pas de velo ici",


    def rendre_velo(self, station):        
        import time
        if not self.sur_velo:
            if self.debug : print "pas en chemin",
            return 0
        if time.time() > self.heure_d_arrivee:
            if station.rendre_velo(self.velo, self.nb_km_trajet):
                
                #if self.debug : print "plus sur velo",
                duree_constatee = self.heure_d_arrivee - self.heure_de_depart
                trajet          = (self.heure_de_depart, self.heure_d_arrivee, self.identifiant, self.nb_km_trajet, self.velo.nom )
                self.log_rendu()
                self.last_rendu = time.time()
                self.sur_velo  = False
                self.nb_trajet = self.nb_trajet + 1
                self.velo      = False
                self.trajets.append(trajet)
                
            else:
                if self.debug : print "impossible de rendre sur station %s"%(station),
                pass
        else:
            if self.debug : print "pas encore arrive",
            pass

# -1) Velo

In [3]:
class Velo:

    def log_reparation(self):
        import time
        reparation =(self.nom,time.time(),self.performance)
        path = "./logs/velo_reparations.csv"
        with open(path, "a") as log_reparation:
            to_print = "%s,%s,%s\n"%reparation
            log_reparation.write(to_print)

    
    def __init__(self, nom, station):
        from scipy.stats import norm
        import time 
        self.nom          = nom
        performance       = norm( loc =0.9, scale=0.2 )
        self.performance  = performance.rvs(1)[0]
        self.station      = [station]
        self.problemes    = []
        self.degradations = []
        self.performances = []
        self.debug        = False
        self.nb_km_trajet = []
        self.heures_rendu = []
        self.disponible   = True


    def log_etat(self):
        import time
        path     = "./logs/velo_%s.txt"%self.nom
        path     = "./logs/velos.csv"
        to_print = "%s,%s,%s,%s,%s,%s\n"%(self.nom              ,
                                           self.heures_rendu[-1] ,  
                                           self.station[-1].nom      ,
                                           self.performances[-1] ,
                                           self.degradations[-1] ,
                                           self.problemes[-1]    )
        with open(path, "a") as log_velo:
            log_velo.write(to_print)
    
    def rendu(self, station, nb_km_trajet):
        import time
        self.disponible = True
        if self.debug : print("velo %s rendu sur station %s"%(self, station))
            
        problemes       = [0] * 50
        problemes_bis   = {"pedale" : 0.2, "roue"  : 1 , "degonfle" : 0.3, "freins" : 0.05 , 0 : 0}
        problemes.extend(problemes_bis.keys())
        probleme        = random.choice(problemes)
        
        self.problemes.append(probleme)
        degradation = sum([ problemes_bis[x] for x in self.problemes if x in problemes_bis.keys()])
        
        self.performance = self.performance - self.performance*degradation
                                        
        self.performances.append(self.performance)
        self.degradations.append(degradation)
        self.nb_km_trajet.append(nb_km_trajet)
        self.station.append(station)
        self.heures_rendu.append(time.time())
        self.log_etat()
        
        if 900 < random.randint(0, 1000 ) :
            self.performance  = 1
            self.log_reparation()
        
        

    def __str__(self):
        return "%s"%self.nom



# -1 ) Station

In [4]:
class Station:
    def __init__(self, nom, debug=True ):
        import random
        import time
        debut         = time.time()
        self.nom      = nom
        self.nb_plot  = random.choice([10,15,20, 30])
        self.nb_libre = 0 #random.randint(0, self.nb_plot )
        self.nb_velos = self.nb_plot # - self.nb_libre
        self.velos    = []
        noms          = get_names()
        
        for n in range(self.nb_velos ):
            nom_velo="velo_"+"".join(["%s"%x for x in random.choice(noms)])
            self.velos.append(Velo(nom_velo , self))
        
        if debug : print "created %s velos"%len(self.velos),
        self.debug          = False
        self.avaries        = []
        self.disponibilites = []
        self.log_dispo()
        self.arrete         = False
        duree               = time.time() - debut
        if debug : print "station créée en %s"%duree


    def log_reparation(self):
        import time
        dispo =(self.nom,time.time(),self.nb_plot, self.nb_libre, self.nb_velos)

        self.disponibilites.append(dispo)
        path = "./logs/stations_reparations.csv"
        with open(path, "a") as log_station:
            to_print = "%s,%s,%s,%s,%s\n"%dispo
            log_station.write(to_print)
            
            
    def log_dispo(self):
        import time
        dispo =(self.nom,time.time(),self.nb_plot, self.nb_libre, self.nb_velos)

        self.disponibilites.append(dispo)
        path = "./logs/station_%s.txt"%self.nom
        path = "./logs/station_.csv"
        with open(path, "a") as log_station:
            to_print = "%s,%s,%s,%s,%s\n"%dispo
            log_station.write(to_print)
            
    def log_avarie(self):
        import time
        avarie = self.avaries[-1][:2]
        heure  = avarie[0]
        pbm    = avarie[1]
        path   = "./logs/station_%s_avarie.txt"%self.nom
        path   = "./logs/avaries.txt"
        with open(path, "a") as f:
            to_print = "%s,%s,%s\n"%(self.nom, heure, pbm )
            f.write(to_print)

    def avarie_possible(self):
        import math
        import random
        import time
        avaries     = [0]*100000
        plot        = "plot mort"
        half        = "50% mort"
        network     = "reseau"
        electricite = "eteint"
        bug         = "bug"
        problemes   = { plot       : 20 ,
                       half        : 10 ,
                       network     :  5 ,
                       electricite : 30 , 
                       bug         :  3 }
        
        for k,v in problemes.iteritems():
            avaries.extend([k] * v)
            
        if plot in self.avaries : avaries.extend( [plot] * 60 )
        if half in self.avaries : avaries.extend( [half] *100 )
        
        avarie = random.choice(avaries)
        self.avaries.extend([avarie])
        if avarie == plot :
            self.nb_libre = self.nb_libre -1
        elif avarie == half:
            self.nb_libre = math.floor(self.nb_libre - (self.nb_plot / float(2)))
        elif avarie == network :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == electricite :
            self.nb_libre = 0
            self.arrete = True
        elif avarie == bug :
            if self.nb_libre >0:
                self.nb_libre = self.nb_libre - random.randint(0, self.nb_libre)
        if self.nb_libre <0:
            self.nb_libre = 0            
        self.log_dispo()
        #self.avaries.append((time.time(),avarie,  self.nb_libre, self.nb_velos))
        if avarie:
            self.log_avarie()
            
    def rendre_velo(self, velo, nb_km_trajet):
        import time
        self.avarie_possible()
        if self.nb_libre >0:
            self.velos.append(velo)
            velo.rendu(self, nb_km_trajet)
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre - 1
            self.log_dispo()
            return 1
        if 9500 < random.randint(0, 10000 ) :
            self.nb_libre = self.nb_plot - self.nb_velos
            self.log_reparation()
            
        return 0

    def prendre_velo(self):
        """
        retourne un vélo ou 0 en cas d'erreur.
        """
        import time
        self.avarie_possible()
        if self.velos and not self.arrete :
            to_return     = self.velos.pop()
            self.nb_velos = len(self.velos)
            self.nb_libre = self.nb_libre + 1
            self.log_dispo()
            return to_return

        return 0
    def __str__(self):
        return "station %s : self.nb_libre = %s, self.nb_velos =%s "%(self.nom,self.nb_libre, self.nb_velos)


# 0) get_names

In [5]:
def get_names(nb_lettres=10, n = 100000, prefix=""):
    """
    retourne n mot de nb_lettres avec prefix.
    """
    import itertools
    from random import shuffle
    lettres = "azertyuiopqsdfghjklmwxcvbn1234567890"
    names   = []
    for i in itertools.combinations(lettres, nb_lettres):
        name = "".join(i)
        name = prefix + name
        names.append(name)
        n = n -1
        if n <2:
            break
    shuffle(names)
    return names

# 0) créatoin des stations

In [6]:

import multiprocessing
n = 4
debut = time()
process_pool = multiprocessing.Pool(n)
debut        = time()
names        = get_names()
noms         = [names[i] for i in range(100)]
stations     = process_pool.map(Station, noms)
nb_velos     = sum([len(station.velos) for station in stations])
duree = time() - debut
print "duree = %s pour %s cree soit %s sec / unite"%(duree, nb_velos, duree/nb_velos)


created 15 velos station créée en 0.197919845581
created 15 velos station créée en 0.208270788193
created 30 velos station créée en 0.213041782379
created 30 velos station créée en 0.228250980377
created 20 velos station créée en 0.231049060822
created 10 velos station créée en 0.215714931488
created 30 velos station créée en 0.250278949738
created 30 velos station créée en 0.242015123367
created 10 velos station créée en 0.215361118317
created 10 velos station créée en 0.221210002899
created 10 velos station créée en 0.210819005966
created 30 velos station créée en 0.252722024918
created 10 velos station créée en 0.244834899902
created 30 velos station créée en 0.249395847321
created 15 velos station créée en 0.249979972839
created 10 velos station créée en 0.234250068665
created 10 velos station créée en 0.235646009445
created 30 velos station créée en 0.241883993149
created 20 velos station créée en 0.242309808731
created 30 velos station créée en 0.261653900146
created 15 velos sta

# Creation des cycliste

In [7]:
n = 4
process_pool = multiprocessing.Pool(n)
noms         = [names[i] for i in range(nb_velos*2)]
cyclistes   = process_pool.map(Cycliste, noms)
print "duree = %s pour %s cree soit %s sec / unite"%(duree, len(cyclistes), duree/nb_velos)


duree = 7.35750794411 pour 3870 cree soit 0.00380232968688 sec / unite


# voyages

In [8]:
from time import time
import multiprocessing
import math
from time import time

In [ ]:

n = 0  
while n <2000 : 
    debut     = time()
    old_nb_trajet = sum([cycliste.nb_trajet for cycliste in cyclistes])
    
    en_route = filter(lambda cycliste:     cycliste.sur_velo, cyclistes)
    a_pied   = filter(lambda cycliste: not cycliste.sur_velo, cyclistes)
    
    map(lambda cycliste: cycliste.rendre_velo(random.choice(stations)), en_route  )
    map(lambda cycliste: cycliste.prend_velo(random.choice(stations)),  a_pied    )
        
    new_nb_trajet =  sum([cycliste.nb_trajet for cycliste in cyclistes]) - old_nb_trajet

    duree_totale = time()-debut
    print "(%03d) %03d / %s'' | "%(n, new_nb_trajet, duree_totale),

    n += 1
print "*"*30
print "fini"
print "*"*30

 (000) 169 / 1.17935395241'' |  (001) 162 / 1.07167482376'' |  (002) 166 / 0.971477031708'' |  (003) 155 / 0.943682909012'' |  (004) 157 / 0.865885972977'' |  (005) 130 / 0.793428897858'' |  (006) 142 / 0.741847991943'' |  (007) 116 / 0.688325881958'' |  (008) 116 / 0.784890174866'' |  (009) 104 / 0.67285490036'' |  (010) 108 / 0.662881135941'' |  (011) 087 / 0.590481996536'' |  (012) 114 / 0.614245891571'' |  (013) 109 / 0.5983710289'' |  (014) 088 / 0.550055027008'' |  (015) 086 / 0.573343992233'' |  (016) 092 / 0.573980093002'' |  (017) 086 / 0.489923000336'' |  (018) 067 / 0.493633031845'' |  (019) 076 / 0.565892934799'' |  (020) 083 / 0.499979019165'' |  (021) 083 / 0.552288770676'' |  (022) 085 / 0.483480930328'' |  (023) 083 / 0.506901025772'' |  (024) 081 / 0.519005060196'' |  (025) 081 / 0.514245033264'' |  (026) 075 / 0.596187114716'' |  (027) 066 / 0.506186962128'' | 